In [ ]:
import astropy
from astropy.wcs import WCS
from astropy import wcs
from astropy.io import fits, ascii
from astropy.coordinates import Angle, SkyCoord
from astropy.nddata import Cutout2D
import astropy.units as un
import numpy as np
import pandas as pd

import matplotlib
from matplotlib.patches import Ellipse, Rectangle
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.ticker import MultipleLocator, FixedFormatter

import current.plot_fqns
from current.plot_fqns import beam, load, info, cut, scale, beam_plotter, sbar_plotter
import warnings
warnings.filterwarnings("ignore");

#### functions

In [ ]:
def beam(hdu):
    beam_dict = {};
    beam_dict = {"b_major": hdu.header['BMAJ'], "b_minor": hdu.header['BMIN'], "b_pa": hdu.header['BPA']};
    #All units in degree
    
    return(beam_dict)


def info(hdu, in_center_dict=None):
    
    if (str(hdu.__class__) == "<class 'astropy.io.fits.hdu.image.PrimaryHDU'>"):
        
        info_dict = {};
        info_dict = { "obs_code": hdu.header['OBSERVER'], "obs_date": hdu.header['DATE-OBS'],
                 "obsra": hdu.header['OBSRA'], "obsdec": hdu.header['OBSDEC'],
                 "rms": hdu.header['NOISE'], "min": hdu.header['DATAMIN'], "max": hdu.header['DATAMAX']};
        center_dict = {};
        center_dict = {"centpxl_ra": hdu.header['CRPIX1'], "centpxl_dec": hdu.header['CRPIX2'],
                "center_ra": hdu.header['CRVAL1'], "center_dec": hdu.header['CRVAL2'],
                "scale_ra": hdu.header['CDELT1'], "scale_dec": hdu.header['CDELT2'],
                "npxl_ra": hdu.header['NAXIS1'], "npxl_dec": hdu.header['NAXIS2']};
        return(info_dict, center_dict);
        
    elif (str(hdu.__class__) == "<class 'astropy.io.fits.header.Header'>"):
        
        info_dict = {};
        info_dict = { "obs_date": hdu['DATE-OBS']};
        center_dict = in_center_dict;
        center_dict |= {"centpxl_ra": hdu['CRPIX1'], "centpxl_dec": hdu['CRPIX2'],
                "center_ra": hdu['CRVAL1'], "center_dec": hdu['CRVAL2'],
                "scale_ra": hdu['CDELT1'], "scale_dec": hdu['CDELT2']};
        return(info_dict, center_dict);
    else:
        print("Not work!",str(hdu.__class___));
    

def load(filename):
    hdu = fits.open(filename)[0];
    hdu_data = hdu.data[0][0];
    wcs_tmp = WCS(hdu.header);
    wcs_tmp1 = wcs_tmp.dropaxis(3);
    wcs = wcs_tmp1.dropaxis(2);
    
    return(hdu_data, wcs, hdu)

def cut(data, in_wcs, hdu, center_dict, center_pos, box):
    
    size = un.Quantity(box, un.pix);
    
    cutout = Cutout2D(data, position=center_pos, size=size, wcs=in_wcs);
    cutout_wcs = cutout.wcs.to_header();
    wcs_cut = WCS(cutout_wcs);
    cutout_data = cutout.data;
    
    center_tmp = cutout.center_cutout;
    center_coord = wcs_cut.pixel_to_world(center_tmp[0],[1]);
    _ , new_center_dict = info(cutout_wcs, center_dict);
    new_center_dict |= {'centpxl_ra': center_tmp[0], 'centpxl_dec':center_tmp[1],
                      'center_ra':center_pos.ra.degree, 'center_dec':center_pos.dec.degree,
                       'npxl_ra': size[0].value, 'npxl_dec':size[1].value};
    
    return(cutout_data, wcs_cut, hdu, new_center_dict)

def scale(hdu, beam_dict, size):
    b_conv = (3600*1e3);
    beamsize = (beam_dict['b_major']*b_conv);
    sbar_scale = (size/beamsize);
    width = (beam_dict['b_major']*sbar_scale);

    return(width);

In [ ]:
# ploting functions
def beam_plotter(beam_dict, center_dict, ax_name):
    
    ra_offset = (center_dict['npxl_ra']/2); #35
    dec_offset = (center_dict['npxl_dec']/2 ); #2
    
    
    beam_ra = (center_dict['center_ra'] - ra_offset*center_dict['scale_ra'] - beam_dict['b_major']/2); 
    beam_dec = (center_dict['center_dec'] - dec_offset*center_dict['scale_dec'] + beam_dict['b_major']);
    
    beam = Ellipse((beam_ra, beam_dec), height=beam_dict['b_major'], width = beam_dict['b_minor'], 
                   angle=-beam_dict['b_pa'], transform=ax_name.get_transform('fk5'), edgecolor='w',
                   facecolor='w', alpha=0.85, lw=2); #angle needs to be neg as matplotlib angle is CCW
    return(beam)

def sbar_plotter(center_dict, beam_dict, sbar_width_ang, ax_name, finetune_ra=0, finetune_dec=0):
    
    b_conv = (3600*1e3);
    beamsize = (beam_dict['b_major']*b_conv);
    sbar_scale = (sbar_width_ang/beamsize);
    
    sbarwidth = (beam_dict['b_major']*sbar_scale);
    
    sbar_ra_offset = (center_dict['npxl_ra']/2 - 20);
    sbar_dec_offset = (center_dict['npxl_dec']/2 - 15);
    
    sbar_rad = (center_dict['center_ra'] + sbar_ra_offset*center_dict['scale_ra']);
    sbar_dec = (center_dict['center_dec'] - sbar_dec_offset*center_dict['scale_dec']);
    
    
    sbartxt = str(sbar_width_ang) + ' mas';
  
    sbar_ra_txt = (sbar_ra_offset - 5.25*sbar_width_ang+ finetune_ra);
    sbar_dec_txt = (sbar_dec_offset- 7 + finetune_dec);
    
    sbar_txt_rad = (center_dict['center_ra'] +
                    sbar_ra_txt*center_dict['scale_ra']);
    sbar_txt_dec = (center_dict['center_dec'] - sbar_dec_txt*center_dict['scale_dec']);

    sbar = Rectangle((sbar_rad, sbar_dec), width = sbarwidth, height = beam_dict['b_minor']/10,
                    transform=ax_name.get_transform('fk5'), color='w', facecolor='w', lw=2);
                     
    return(sbar, sbartxt, sbar_txt_rad, sbar_txt_dec);

def open(filename):
    img_data, wcs, image = load(filename);
    
    img_info = info(image);
    beam_data = beam(image);
    
    allinone = (img_info | beam_data );
    return(img_data, wcs, image, allinone)

### Aips fit

In [ ]:

    
def fit_plotter(name, ax_name):
    df = pd.read_csv(name, delimiter='\s\s+', engine='python', header=[0,1]);
    coords = SkyCoord(df.iat[0,3], df.iat[0,4], unit=(un.hourangle, un.degree), frame='fk5');
    theta = -df.iat[0,7];
    maj = ((df.iat[0,5])*un.mas).to(un.degree);
    min = ((df.iat[0,6])*un.mas).to(un.degree);
    fitra = coords.ra.degree;
    fitdec = coords.dec.degree;
    # fit = Ellipse((fitra, fitdec), height=maj, width=min, angle=theta,
    #               transform=ax_name.get_transform('fk5'), edgecolor='w', facecolor='w', 
    #               alpha=0.5, lw=2);
    fit = Ellipse((fitra, fitdec), height=maj.value, width = min.value, 
                   angle=theta, transform=ax_name.get_transform('fk5'), edgecolor='w',
                   facecolor='w', alpha=0.3, lw=2);
    return(coords, fit);

#### Gaia

In [ ]:
gaia_df = pd.read_csv('./tables/V1674HER_gaia.csv', delimiter=',');
v1674her_df = gaia_df.iloc[3];
v1674her_gaia = v1674her_df.to_dict();
#ra_pm = un.Quantity(v1674her_gaia['pmra'], unit=(un.mas/un.year));

gaia = SkyCoord(v1674her_gaia['ra'], v1674her_gaia['dec'], unit=(un.degree, un.degree), 
                obstime="J2016", pm_ra_cosdec=(v1674her_gaia['pmra']*(un.mas/un.year)), 
                pm_dec=(v1674her_gaia['pmdec']*(un.mas/un.year)));

ra_err = (v1674her_df['dec_error']*un.mas).to(un.degree).value;
dec_err = (v1674her_df['ra_error']*un.mas).to(un.degree).value;
rapm_err = (v1674her_df['pmra_error']*un.mas).to(un.degree).value;
decpm_err = (v1674her_df['pmdec_error']*un.mas).to(un.degree).value;

gaia_xerr = (ra_err**2+rapm_err**2)**(1/2);
gaia_yerr = (dec_err**2+rapm_err**2)**(1/2);
# shifted_gaia_tmp = gaia.apply_space_motion(new_obstime=astropy.time.Time(imgb_info['obs_date']));
# shifted_gaia = shifted_gaia_tmp.transform_to('fk5'); #AIPS uses FK5
gaia

#### Difmap Center from V1674HERB

In [ ]:
old_difmap = SkyCoord(284.3790725790, 16.8943023350, unit=(un.degree, un.degree), 
                obstime="J2000", frame='fk5'); #the coordinates come from the difmap fit
difmap = SkyCoord(284.3790727910, 16.8943022807, unit=(un.degree, un.degree), 
                obstime="J2000", frame='fk5');
sep = (difmap.separation(gaia)).to_string(unit=un.degree, sep=':');
difmap.to_string('hmsdms')

In [ ]:
difmapd = SkyCoord(284.3790728251, 16.8943023740, unit=(un.degree, un.degree),
                   obstime="J2000", frame='fk5');
difmap_nd = SkyCoord(284.3790728379, 16.8943024352, unit=(un.degree, un.degree), 
                     obstime="J2000", frame='fk5');
# print(difmap_nd.to_string('hmsdms'));

In [ ]:
#fitted from difmap
major = (3.417e-02*un.arcsec).to(un.degree);
minor = (1.515e-02*un.arcsec).to(un.degree);
theta = 50.24*un.degree


### RMS

In [ ]:
b_rms = 1.69e-02 #mJy/beam
d_rms = 1.38e-02
ft_rms = 1.291e-02
fnt_rms = 1.323e-02
h_rms = 7.409e-03
ft_peak = 26.678e-03
fnt_peak = 24.192e-03
ft1_rms = 1.316e-02
ft1_peak = 4.9394e-02
fnt1_rms = 1.587e-02
fnt1_peak = 5.933e-02

amount_ft = ft1_peak/ft1_rms
amount_fnt = fnt1_peak/fnt1_rms



### V1674HERB

In [ ]:
imgb_data, wcsb, imageb = load('./data/020524/V1674HERB_CLN2.fits');

imgb_info, centerb = info(imageb);
beamb = beam(imageb);
difmap
#tst_data, tst_wcs, tst_img, tst_aio = open('./V1674HERIMG/V1674HERB_CLN2.fits');
wcsb.keys()

In [ ]:
cutb_data, cut_wcsb, cutb_hdu, cutb_center = cut(imgb_data, wcsb, imageb, centerb, center_pos=difmap, box=(350,350));


shifted_gaia_tmp = gaia.apply_space_motion(new_obstime=astropy.time.Time(imgb_info['obs_date']));

shifted_gaia = shifted_gaia_tmp.transform_to('fk5');


In [ ]:
sep = shifted_gaia.separation(difmap);
conv = (3600)**(-1)*(np.pi/180) #arcsec to rad
d = 6.3e3
theta = (10e-3)*conv
sep_dis = np.tan(sep.rad)*d
pc_au = 2.0626481e5
sep_au = sep_dis*pc_au
# f'{D_au:.2E}'
f'{sep.mas:.2E}' #169 AU or 27mas off

In [ ]:
""" beamb """
print(plt.style.available)

In [ ]:
#intial stuff

plt.rcParams.update({'font.size': 19});
plt.rcParams.update({'font.weight': 'heavy'});
plt.rcParams.update({'text.color': 'white'});

# plt.rcParams.update({
#     "text.usetex": False,
#     "font.family": "sans-serif",
#     "font.family": "Helvetica"
# });

figb = plt.figure(figsize=(17,13), facecolor='black');
axb = figb.add_axes([0.2, 0.2, 0.8, 0.8], projection=cut_wcsb);
ra_b = axb.coords[0];
dec_b = axb.coords[1];

axb.set_title(f'V1674HER \n 24 Days After Eruption', fontsize=25, pad=10, color='white', fontweight='heavy');

#axes ticks
dec_b.display_minor_ticks(True);
ra_b.display_minor_ticks(True);
axb.tick_params(which='both', direction='in', color='white', length=17, width=2, pad=10.0);
axb.tick_params(which='minor', length=7);


imgb = axb.imshow(cutb_data, cmap='inferno', vmin=imgb_info['min'], vmax=imgb_info['max'], 
                 transform=axb.get_transform(cut_wcsb), origin='lower');


contour_levels_array = np.array((-1.5, 2, 5, 7, 9, 11));
contour_levels = imgb_info['rms']*contour_levels_array;
# for spine in imgb.axes.spines.values():
#     spine.set_edgecolor('white');

cbar_b = figb.colorbar(imgb, location='right', spacing='proportional', format='{x:^8.2e}'); 
cbar_b.ax.set_ylabel('Jy/beam', labelpad=13.0, fontweight='heavy', color='white'); 
cbar_b.ax.yaxis.set_tick_params(color='white');
cbar_b.outline.set_edgecolor('white');
plt.setp(plt.getp(cbar_b.ax.axes, 'yticklabels'), color='white')

ra_b.set_axislabel('RA (J2000)', minpad=1.5, fontweight='heavy', color ='white');
dec_b.set_axislabel('DEC (J2000)', minpad=1.0, fontweight='heavy', color='white');
ra_b.set_ticklabel('white');
dec_b.set_ticklabel('white');
axb.coords.frame.set_color('white')
# dec_b.label.set_color('white')
plt.rcParams.update({'lines.linewidth': 1.5});

axb.contour(cutb_data, levels=contour_levels ,transform=axb.get_transform(cut_wcsb), colors='w', alpha=0.6);

scale_bar, sbartxt, sbar_txt_rad, sbar_txt_dec = sbar_plotter(cutb_center, beamb, 10, axb);
axb.text(sbar_txt_rad, sbar_txt_dec, sbartxt, transform=axb.get_transform('fk5'), color='w', fontweight='bold');

beam_patch = beam_plotter(beamb, cutb_center, axb);
hi = fit_plotter('./imfit/v1674herb_imfit.txt', axb);
plt.rcParams.update({'axes.labelcolor': 'white'});
plt.rcParams.update({'xtick.labelcolor': 'white'});
plt.rcParams.update({'axes.labelsize': 'large'});
# axb.add_patch(hi)
conv = (3600)**(-1);
aips_fit_coord = SkyCoord('18 57 30.9776461', '16 53 39.491168', unit=(un.hourangle, un.degree), frame='fk5');
fit_height = 0.0165906 *conv
fit_width = 0.0080879 *conv
aips_fit = Ellipse((aips_fit_coord.ra.degree, aips_fit_coord.dec.degree), height=fit_height, width=fit_width, angle=-138.275, transform=axb.get_transform('fk5'), facecolor='w');
# axb.add_patch(aips_fit);
axb.plot(aips_fit_coord.ra.degree, aips_fit_coord.dec.degree,  transform=axb.get_transform('fk5'), marker='*');
# fit = Ellipse((284.3790735331, 16.8943031878), height=2.9833333333333333e-06, width = 2.648611111111111e-06, 
#                    angle=69.32, transform=axb.get_transform('fk5'), edgecolor='w',
#                    facecolor='w', alpha=0.85, lw=2);
# axb.add_patch(aips_fit);
# axb.add_patch(beam_patch);
# axb.add_patch(scale_bar);


axb.errorbar(shifted_gaia.ra.degree, shifted_gaia.dec.degree, yerr=gaia_yerr, xerr=gaia_xerr, transform=axb.get_transform('fk5'), 
             markersize=15, color='yellow', marker='.', label='Gaia position');
# axb.plot(centerb_new['center_ra'], centerb_new['center_dec'], transform=axb.get_transform('fk5'), 
#          markersize=15, color='white', alpha=0.7, marker='*', label='Gaia position');

axb.plot(difmap.ra.degree, difmap.dec.degree, transform=axb.get_transform('fk5'), 
             markersize=17, color='purple',  marker='*',label='Difmap Center Position \n from day 20');
axb.legend(labelcolor='black', facecolor ='white', framealpha=0.6);

### V1674HERD

In [ ]:
imgd_data, wcsd, imaged = load('./data/020524/V1674HERD_CLN.fits');

imgd_info, centerd = info(imaged);
beamd = beam(imaged);


cutd_data, cut_wcsd, cutd_hdu, cutd_center = cut(imgd_data, wcsd, imaged, centerd, center_pos=difmap, box=(350,350));

sbar_widthd = scale(cutd_hdu, beamd, 10);
imgd_info['rms']

In [ ]:
imgd_info

In [ ]:
figd = plt.figure(figsize=(17,13), facecolor='black');
axd = figd.add_axes([0.2, 0.2, 0.8, 0.8], projection=cut_wcsd); # slices=('x','y',1,1)
ra_d = axd.coords[0];
dec_d = axd.coords[1];

axd.set_title(f'V1674HER \n 24 Days After Eruption', fontsize=25, pad=10, fontweight='heavy');

#axes ticks
dec_d.display_minor_ticks(True);
ra_d.display_minor_ticks(True);
axd.tick_params(which='both', direction='in', color='white', length=13, width=1);
axd.tick_params(which='minor', length=5);


imgd = axd.imshow(cutd_data, cmap='inferno',  vmin=imgb_info['min'], vmax=imgd_info['max'], 
                 transform=axd.get_transform(cut_wcsd), origin='lower');

cbar_d = figd.colorbar(imgd, location='right', format='{x:^8.2e}');
cbar_d.ax.set_ylabel(r'Jy/beam', fontweight='heavy');

ra_d.set_axislabel('RA (J2000)', minpad=1, fontweight='heavy');
dec_d.set_axislabel('DEC (J2000)', minpad=-1.0, fontweight='heavy');

beam_patchd = beam_plotter(beamd, cutd_center, axd);
scale_bard, sbardtxt, sbard_txt_ra, sbard_txt_dec = sbar_plotter(cutd_center, beamd, 10, axd);

axd.add_patch(beam_patchd);
contour_levelsd_array = np.array((-1.5, 2, 5, 7, 9, 11));
contour_levelsd = imgd_info['rms']*contour_levelsd_array;

# axd.add_patch(scale_bard);
# axd.text(sbard_txt_ra, sbard_txt_dec, sbardtxt, transform=axd.get_transform('fk5'), color='w', weight='bold');

axd.contour(cutd_data, levels=contour_levelsd ,transform=axd.get_transform(cut_wcsd), colors='w', alpha=0.7);
# plotting countours from b on d
plt.rcParams["lines.linewidth"] = 1.5;
cont_levels_array = np.array((3, 5, 7, 9, 11));
# cont_levels = imgb_info['rms']*cont_levels_array;
# axd.contour(cutb_data, levels=cont_levels ,transform=axd.get_transform(cut_wcsd), colors='w', alpha=0.5);
# fit minor=1.15E-06 major=3.86E-06
axd.errorbar(shifted_gaia.ra.degree, shifted_gaia.dec.degree, yerr=gaia_yerr, xerr=gaia_xerr, transform=axd.get_transform('fk5'), 
         markersize=9, color='white', marker='.', label='Gaia position');

axd.plot(difmap.ra.degree, difmap.dec.degree, transform=axd.get_transform('fk5'), 
         markersize=15, color='purple', marker='*', alpha=0.65, label='V1674HER position');
aipsd_coord, _ = fit_plotter('./imfit/v1674herd_imfit.txt', axd);

# fit = Ellipse((284.3790735331, 16.8943031878), height=3.86E-06, width = 1.15E-06, 
#                    angle=44.73, transform=axd.get_transform('fk5'), edgecolor='w',
#                    facecolor='w', alpha=0.85, lw=2);
# axd.add_patch(fit);
# axd.plot(difmapd.ra.degree, difmapd.dec.degree, transform=axd.get_transform('fk5'), 
#          markersize=15, color='cornflowerblue', marker='*', alpha=0.65, label='V1674HER position');

# axd.plot(aipsd_coord.ra.degree, aipsd_coord.dec.degree, transform=axd.get_transform('fk5'), 
#          markersize=15, color='purple', marker='*', alpha=0.75, label='AIPS Fit Center');

# axd.legend(framealpha=0.85);

### V1674HERF

In [ ]:
imgf_data, wcsf, imagef = load('./data/020524/V1674HERF_CLN.fits');

imgf_info, centerf = info(imagef);
beamf = beam(imagef);


cutf_data, cutf_wcs, cutf_hdu, cutf_center = cut(imgf_data, wcsf, imagef, centerf,
                                    center_pos=difmap, box=(350,350));



sbarf_width = scale(cutf_hdu, beamf, 10);
imgf_info

In [ ]:
figf = plt.figure(figsize=(17,13), facecolor='black');
axf = figf.add_axes([0.2, 0.2, 0.8, 0.8], projection=cutf_wcs);
ra_f = axf.coords[0];
dec_f = axf.coords[1];

axf.set_title(f'V1674HER \n 35 Days After Eruption \n Tapered', fontsize=20);

dec_f.display_minor_ticks(True);
ra_f.display_minor_ticks(True);
axf.tick_params(which='both', direction='in', color='white', length=13, width=1);
axf.tick_params(which='minor', length=5);

imgf = axf.imshow(cutf_data, cmap='inferno', vmin=imgf_info['min'], vmax=imgf_info['max'],
                  transform=axf.get_transform(cutf_wcs), origin='lower');

cbar_f = figf.colorbar(imgf, location='right');
cbar_f.ax.set_ylabel(r'$\mu$Jy/beam');

ra_f.set_axislabel('RA (J2000)', minpad=1);
dec_f.set_axislabel('DEC (J2000)', minpad=-1.0);

beam_patchf = beam_plotter(beamf, cutf_center, axf);
scale_barf, sbarftxt, sbarf_txt_ra, sbarf_txt_dec = sbar_plotter(cutf_center, beamf, 10, axf);
aipsft_coord, _ = fit_plotter('./imfit/v1674herft_imfit.txt', axf);

axf.add_patch(beam_patchf);
axf.add_patch(scale_barf);

axf.errorbar(shifted_gaia.ra.degree, shifted_gaia.dec.degree, yerr=gaia_yerr, xerr=gaia_xerr, transform=axf.get_transform('fk5'), 
         markersize=9, color='white', marker='.', label='Gaia position');
axf.text(sbarf_txt_ra, sbarf_txt_dec, sbarftxt, transform=axf.get_transform('fk5'), color='w', weight='bold');

axf.plot(difmap.ra.degree, difmap.dec.degree,  transform=axf.get_transform('fk5'), 
         markersize=15, color='white', marker='*', alpha=0.65, label='V1674HER position');
# axf.plot(aipsft_coord.ra.degree, aipsft_coord.dec.degree, transform=axf.get_transform('fk5'), 
#          markersize=15, color='purple', marker='*', alpha=0.65, label='AIPS Fit Center');
axf.legend();

contour_levelsf_array = np.array((-1.5, 2, 5, 7, 9, 11));
contour_levelsf = imgf_info['rms']*contour_levelsf_array;
axf.contour(cutf_data, levels=contour_levelsf ,transform=axf.get_transform(cutf_wcs), colors='w', alpha=0.7);


In [ ]:
aipsft_coord.separation(difmap).mas
V1674HER_dif = SkyCoord('18:57:30.977', '16:53:39.499', unit=(un.hourangle, un.deg), frame='fk5');
V1674HER_man = SkyCoord('18:57:30.979', '16:53:39.535', unit=(un.hourangle, un.deg), frame='fk5');
aipsft_coord.separation(V1674HER_dif).mas
print(aipsft_coord.ra.hms, aipsft_coord.dec.dms)


#### Non-Tapered

In [ ]:
imgj_data, wcsj, imagej = load('./V1674HERIMG/V1674HERF_CLN2_NOT.fits');

imgj_info, centerj = info(imagej);
beamj = beam(imagej);


cutj_data, cutj_wcs, cutj_hdu, cutj_center = cut(imgj_data, wcsj, imagej, centerj,
                                    center_pos=difmap, box=(350,350));



sbarj_width = scale(cutj_hdu, beamj, 10);


In [ ]:
#intial stuff
plt.rcParams.update({'font.size': 12});


figj = plt.figure(figsize=(17,11));
axj = figj.add_axes([0.2, 0.2, 0.8, 0.8], projection=cutj_wcs);
ra_j = axj.coords[0];
dec_j = axj.coords[1];

axj.set_title(f'V1674HER \n 35 Days After Eruption \n Not Tapered', fontsize=20);

#axes ticks
dec_j.display_minor_ticks(True);
ra_j.display_minor_ticks(True);
axj.tick_params(which='joth', direction='in', color='white', length=13, width=1);
axj.tick_params(which='minor', length=5);


imgj = axj.imshow(cutj_data, cmap='inferno', vmin=imgj_info['min'], vmax=imgj_info['max'], 
                 transform=axj.get_transform(cutj_wcs), origin='lower');

cjar_j = figj.colorbar(imgj, location='right');
cjar_j.ax.set_ylabel(r'$\mu$Jy/beam');

ra_j.set_axislabel('RA (J2000)', minpad=1);
dec_j.set_axislabel('DEC (J2000)', minpad=-1.0);

beam_patch = beam_plotter(beamj, cutj_center, axj);
scale_jar, sjartxt, sjar_txt_rad, sjar_txt_dec = sbar_plotter(cutj_center, beamj, 10, axj);
aipsfnt_coord, _ = fit_plotter('./imfit/v1674herfnt_imfit.txt', axj);

axj.add_patch(beam_patch);
axj.add_patch(scale_jar);


axj.text(sjar_txt_rad, sjar_txt_dec, sjartxt, transform=axj.get_transform('fk5'), color='w', weight='bold');
axj.errorbar(shifted_gaia.ra.degree, shifted_gaia.dec.degree, yerr=gaia_yerr, xerr=gaia_xerr, transform=axj.get_transform('fk5'), 
         markersize=9, color='white', marker='.', label='Gaia position');
# axj.plot(centerj_new['center_ra'], centerj_new['center_dec'], transform=axj.get_transform('fk5'), 
#          markersize=15, color='white', alpha=0.7, marker='*', lajel='Gaia position');

axj.plot(difmap.ra.degree, difmap.dec.degree, transform=axj.get_transform('fk5'), 
         markersize=15, color='w', alpha=0.6, marker='*', label='Difmap Center Position');
axj.plot(aipsfnt_coord.ra.degree, aipsfnt_coord.dec.degree, transform=axj.get_transform('fk5'), 
         markersize=15, color='purple', marker='*', alpha=0.65, label='V1674HER position');
axj.legend();

In [ ]:
imgh_data, wcsh, imageh = load('./V1674HERIMG/V1674HERHt_CLN.fits');

imgh_info, centerh = info(imageh);
beamh = beam(imageh);


cuth_data, cuth_wcs, cuth_hdu, cuth_center = cut(imgh_data, wcsh, imageh, centerh,
                                    center_pos=difmap, box=(350,350));



sbarh_width = scale(cuth_hdu, beamh, 10);


In [ ]:
figh = plt.figure(figsize=(17,11));
axh = figh.add_axes([0.2, 0.2, 0.8, 0.8], projection=cuth_wcs);
ra_h = axh.coords[0];
dec_h = axh.coords[1];

axh.set_title(f'V1674HER \n 71 Days After Eruption', fontsize=20);

dec_h.display_minor_ticks(True);
ra_h.display_minor_ticks(True);
axh.tick_params(which='both', direction='in', color='white', length=13, width=1);
axh.tick_params(which='minor', length=5);

imgh = axh.imshow(cuth_data, cmap='inferno', vmin=imgh_info['min'], vmax=imgh_info['max'],
                  transform=axh.get_transform(cuth_wcs), origin='lower');

cbar_h = figh.colorbar(imgh, location='right');
cbar_h.ax.set_ylabel(r'$\mu$Jy/beam');

ra_h.set_axislabel('RA (J2000)', minpad=1);
dec_h.set_axislabel('DEC (J2000)', minpad=-1.0);

beam_patchh = beam_plotter(beamh, cuth_center, axh);
scale_barh, sbarhtxt, sbarh_txt_ra, sbarh_txt_dec = sbar_plotter(cuth_center, beamh, 10, axh);


axh.add_patch(beam_patchh);
axh.add_patch(scale_barh);

axh.errorbar(shifted_gaia.ra.degree, shifted_gaia.dec.degree, yerr=gaia_yerr, xerr=gaia_xerr, transform=axh.get_transform('fk5'), 
         markersize=9, color='yellow', marker='.', label='Gaia position');
axh.text(sbarh_txt_ra, sbarh_txt_dec, sbarhtxt, transform=axh.get_transform('fk5'), color='w', weight='bold');

axh.plot(difmap.ra.degree, difmap.dec.degree,  transform=axh.get_transform('fk5'), 
         markersize=15, color='white', marker='*', alpha=0.65, label='V1674HER position');
# axh.plot(aips_fit_coord.ra.degree, aips_fit_coord.dec.degree, transform=axh.get_transform('fk5'), 
#          markersize=15, color='w', marker='*', alpha=0.65, label='V1674HER position');
axh.legend();

In [ ]:
imgk_data, wcsk, imagek = load('./imfit/img/V1674HERH_NOT.fits');

imgk_info, centerk = info(imagek);
beamk = beam(imagek);


cutk_data, cutk_wcs, cutk_hdu, cutk_center = cut(imgk_data, wcsk, imagek, centerk,
                                    center_pos=difmap, box=(350,350));



sbark_width = scale(cutk_kdu, beamk, 10);


In [ ]:
figh = plt.figure(figsize=(17,11));
axk = figh.add_axes([0.2, 0.2, 0.8, 0.8], projection=cutk_wcs);
ra_h = axk.coords[0];
dec_h = axk.coords[1];

axk.set_title(f'V1674HER \n 71 Days After Eruption', fontsize=20);

dec_h.display_minor_ticks(True);
ra_h.display_minor_ticks(True);
axk.tick_params(which='both', direction='in', color='white', length=13, width=1);
axk.tick_params(which='minor', length=5);

imgh = axk.imshow(cutk_data, cmap='inferno', vmin=imgh_info['min'], vmax=imgh_info['max'],
                  transform=axk.get_transform(cutk_wcs), origin='lower');

cbar_h = figh.colorbar(imgh, location='right');
cbar_h.ax.set_ylabel(r'$\mu$Jy/beam');

ra_h.set_axislabel('RA (J2000)', minpad=1);
dec_h.set_axislabel('DEC (J2000)', minpad=-1.0);

beam_patchh = beam_plotter(beamh, cutk_center, axk);
scale_barh, sbarhtxt, sbarh_txt_ra, sbarh_txt_dec = sbar_plotter(cutk_center, beamh, 10, axk);

axk.add_patch(beam_patchh);
axk.add_patch(scale_barh);

axk.errorbar(shifted_gaia.ra.degree, shifted_gaia.dec.degree, yerr=gaia_yerr, xerr=gaia_xerr, transform=axk.get_transform('fk5'), 
         markersize=9, color='yellow', marker='.', label='Gaia position');
axk.text(sbarh_txt_ra, sbarh_txt_dec, sbarhtxt, transform=axk.get_transform('fk5'), color='w', weight='bold');

axk.plot(difmap.ra.degree, difmap.dec.degree,  transform=axk.get_transform('fk5'), 
         markersize=15, color='white', marker='*', alpha=0.65, label='V1674HER position');
# axk.plot(aips_fit_coord.ra.degree, aips_fit_coord.dec.degree, transform=axk.get_transform('fk5'), 
#          markersize=15, color='w', marker='*', alpha=0.65, label='V1674HER position');
axk.legend();

In [ ]:
V1674HERB_rms =  2.581E-05*un.Jy; #JY/
V1674HERB_max = 1.3560E-04*un.Jy;
V1674HERBp_rms = 2.609E-05*un.Jy;
V1674HERBp_max = 1.3230E-04*un.Jy;

In [ ]:
print(V1674HERBp_max.to(un.uJy));

In [ ]:
V1674HERD_rms = 1.454E-05*un.Jy;
V1674HERD_max = 9.2699E-05*un.Jy;
print(V1674HERD_rms.to(un.uJy))

In [ ]:
imgd_info

In [ ]:
ge